In [2]:
import pandas as pd

### Create Nodes for Mapping

In [3]:
raw = pd.read_csv('../data/finalround/unreducedStops.csv', nrows=1000)
stops_dropped = raw.drop(['Unnamed: 0', 'id'], axis=1)
stops_dropped.head(1)

,type,assigned,mtaTypeStatId,lat,long,mikeStationId
0,tr,0,tr101,40.889377,-73.898421,butr0


### Create Edges for Mapping

In [25]:
trips = pd.read_csv('../data/finalround/busTrainTrips.csv', nrows=1000)
trips_dropped = trips.drop(['Unnamed: 0'], axis=1)

def concatenatorator2(x):
    tmp = str()
    if x == "bikeStop":
        tmp = 'bi'
    elif x == "busStop":
        tmp = 'bu'
    elif x == "trainStop":
        tmp = 'tr'
    return tmp

# use the raw loaded data to create a complete list of all
# mta station ids and mike's assigned ids
# this is different then the groupby approach above.
trips_dropped['type2'] = trips_dropped['type'].apply(concatenatorator2)
trips_dropped['type'] = trips_dropped['type2']
trips_dropped['mta_stop_id'] = trips_dropped['type']+trips_dropped['mta_stop_id'].astype(str)
trips_dropped['mta_start_id'] = trips_dropped['type']+trips_dropped['mta_start_id'].astype(str)
trips_dropped.drop(['type2', 'type'], axis=1, inplace=True)
trips_dropped.head(1)

,mta_stop_id,mta_start_id,duration
0,tr902N,tr901N,1.5


In [ ]:
print stops_dropped[stops_dropped['mtaTypeStatId']=='901N']

In [ ]:
# join the groupby of mikes id and the complete list of mta stations
joined_dfs = trips_dropped.merge(stops_dropped, how='left', left_on='mta_stop_id', right_on='mtaTypeStatId')

joined_dfs.to_csv('/Users/matthewdunn/Dropbox/NYU/Spring2016/BigData/GroupProject/data/prepped_edges.csv', index=False)

In [ ]:
joined_dfs_dropped = joined_dfs.drop(['id', 'type', 'assigned', 'mtaTypeStatId'], axis=1)
joined_dfs_grouped = joined_dfs_dropped.groupby('mikeStationId').mean()
joined_dfs_grouped.head()

In [ ]:
import json
import json
from shapely.geometry import shape, Point
with open('/Users/matthewdunn/Dropbox/NYU/Spring2016/BigData/GroupProject/nyccensustracts.json', 'r') as f:
    js = json.load(f)


def geocoder(lat, lon):
    point = Point(lon, lat)

    # check each polygon to see if it contains the point
    for feature in js['features']:
        geometry = feature.get('geometry')
        polygon = shape(geometry)
        if polygon.contains(point):
                properties = feature.get('properties')
                census_tract = properties.get('BoroCT2010')
                return census_tract

In [ ]:
censustracts = []
for i, row in joined_dfs_grouped.iterrows():
    censustract = geocoder(row.lat, row['long'])
    censustracts.append(censustract)
censustracts

In [ ]:
allpairsBus = pd.read_csv("./data/allPairDistancesBusTrain.csv")
allpairsBus.head()

In [ ]:
allpairsBus_drop = allpairsBus.drop(['assigned', 'Unnamed: 0.1'], 1)
allpairsBus_drop = allpairsBus_drop.drop([0, 1])
cols = list(allpairsBus_drop.columns.values)
cols = cols[1:]
duration_df = pd.melt(allpairsBus_drop, id_vars=['Unnamed: 0'], value_vars=cols)
duration_df.columns = ['mikestartId', 'mikeendId', 'duration']
duration_df = duration_df[duration_df.mikestartId != duration_df.mikeendId]
duration_df.head()

In [ ]:
joined_dfs_grouped['censustract'] = censustracts
joined_dfs_grouped.head()

In [ ]:
merged_duration = duration_df.merge(joined_dfs_grouped, how='left', left_on='mikestartId', right_index=True)
merged_duration.columns = ['mikestartId','mikeendId','duration','start_lat','start_long','start_censustract']
merged_duration.head()

In [ ]:
merged_duration_final = merged_duration.merge(joined_dfs_grouped, how='left', left_on='mikeendId', right_index=True)

In [ ]:
merged_duration_final.columns = ['mikestartId','mikeendId','duration','start_lat','start_long','start_censustract','end_lat','end_long','end_censustract']
merged_duration_final.head()

In [ ]:
from haversine import haversine
def manhattandist(start_lat, start_lon, end_lat, end_lon):
    start_station_location = (start_lat, start_lon)
    end_station_location = (end_lat, end_lon)
    return haversine(start_station_location, end_station_location, miles=True)

In [ ]:
distances = []
for i, row in merged_duration_final.iterrows():
    distance = manhattandist(row.start_lat, row.start_long, row.end_lat, row.end_long)
    distances.append(distance)
len(distances)

In [ ]:
merged_duration_final['distance'] = distances
merged_duration_final.head()

In [ ]:
merged_duration_final.shape

In [ ]:
merged_duration_final.to_csv('/Users/matthewdunn/Dropbox/NYU/Spring2016/BigData/GroupProject/data/mikeCTsAvgLatLonDist.csv')

## Now Create the Bus Sheet for Map Reduce

In [ ]:
merged_raw = pd.read_csv('/Users/matthewdunn/Dropbox/NYU/Spring2016/BigData/GroupProject/data/mta_bus/mergedRoutesStops.csv')

In [ ]:
merged_raw.shape

In [ ]:
trips_dropped = merged_raw.drop(['Unnamed: 0', 'trip_id', 'start_lat', 'start_lon', 'end_lon', 'end_lat', 'arrival_time', 'departure_time'], axis=1)
trips_dropped['end_location_id'] = trips_dropped['end_location_id'].astype(int)
trips_dropped['start_mtaTypeStatId'] = 'bu'+trips_dropped['start_location_id'].astype(str)
trips_dropped['end_mtaTypeStatId'] = 'bu'+trips_dropped['end_location_id'].astype(str)
trips_dropped.drop(['end_location_id', 'start_location_id'], axis=1, inplace=True)
print trips_dropped.head()
trips_dropped.shape

In [ ]:
# Dedupe the Merge Key between MTA and Mikes ID
joined_deduped = joined_dfs.drop_duplicates()
print joined_deduped.head()
# Create a Start Station Version
startStation = joined_deduped[['mtaTypeStatId','lat','long','mikeStationId']].copy()
startStation.columns = ['start_mtaTypeStatId','start_lat','start_long','start_mikeStationId']
# Create an End Station Version
endStation = joined_deduped[['mtaTypeStatId','lat','long','mikeStationId']].copy()
endStation.columns = ['end_mtaTypeStatId','end_lat','end_long','end_mikeStationId']
print endStation.head()

In [ ]:
# validating shit works
# joined_deduped.loc[joined_deduped.id == '901N']
# print any(joined_deduped.mtaTypeStatId == 'bu901N')
# print any(startStation.start_mtaTypeStatId == '901N')

In [ ]:
final = pd.merge(trips_dropped, startStation, on='start_mtaTypeStatId', how='left')
final2 = final.merge(endStation, how='left', left_on='end_mtaTypeStatId', right_on='end_mtaTypeStatId')
final2.shape

In [ ]:
fianl2_dropped = final2.drop(['start_mtaTypeStatId', 'end_mtaTypeStatId', 'start_lat', 'start_long','end_lat', 'end_long'], axis=1)
print fianl2_dropped.head()
print fianl2_dropped.shape

In [ ]:
fianl2_dropped['key'] = fianl2_dropped['start_mikeStationId']+'_'+fianl2_dropped['end_mikeStationId']
mapreduce = fianl2_dropped.drop(['start_mikeStationId', 'end_mikeStationId'], axis=1)
mapreduce = mapreduce[['key', 'duration']]
mapreduce['duration'] = mapreduce['duration']*60
print mapreduce.head()
print mapreduce.shape
mapreduce.to_csv('/Users/matthewdunn/Dropbox/NYU/Spring2016/BigData/GroupProject/data/mta_bus/mapreduce.csv', index=False, header=False)

### Used to write out small sample files

In [ ]:
shorter = pd.read_csv('/Users/matthewdunn/Dropbox/NYU/Spring2016/BigData/GroupProject/data/allPairDistancesBusTrain.csv', nrows=100)

In [ ]:
shorter.to_csv('/Users/matthewdunn/Dropbox/NYU/Spring2016/BigData/GroupProject/data/allPairDistancesBusTrain_100.csv', index=False)

## Now Create the Train Sheet for Map Reduce

In [ ]:
merged_raw_train = pd.read_csv('/Users/matthewdunn/Dropbox/NYU/Spring2016/BigData/GroupProject/data/mta_train/mergedTrainRoutesStops.csv')

In [ ]:
merged_raw_train.shape

In [ ]:
trips_dropped_train = merged_raw_train.drop(['Unnamed: 0', 'trip_id', 'start_lat', 'start_lon', 'end_lon', 'end_lat', 'arrival_time', 'departure_time'], axis=1)
# trips_dropped_train['end_location_id'] = trips_dropped_train['end_location_id'].astype(int)
trips_dropped_train['start_mtaTypeStatId'] = 'tr'+trips_dropped_train['start_location_id'].astype(str)
trips_dropped_train['end_mtaTypeStatId'] = 'tr'+trips_dropped_train['end_location_id'].astype(str)
trips_dropped_train.drop(['end_location_id', 'start_location_id'], axis=1, inplace=True)
print trips_dropped_train.head()
print trips_dropped_train.shape

In [ ]:
any(joined_dfs.mtaTypeStatId == 'tr901N')

In [ ]:
final_train = pd.merge(trips_dropped_train, startStation, on='start_mtaTypeStatId', how='left')
final2_train = final_train.merge(endStation, how='left', left_on='end_mtaTypeStatId', right_on='end_mtaTypeStatId')
final2_train.shape

In [ ]:
fianl2_train_dropped = final2_train.drop(['start_mtaTypeStatId', 'end_mtaTypeStatId', 'start_lat', 'start_long','end_lat', 'end_long'], axis=1)
print fianl2_train_dropped.head()
print fianl2_train_dropped.shape

In [ ]:
fianl2_train_dropped['key'] = fianl2_train_dropped['start_mikeStationId']+'_'+fianl2_train_dropped['end_mikeStationId']
mapreduce_train = fianl2_train_dropped.drop(['start_mikeStationId', 'end_mikeStationId'], axis=1)
mapreduce_train = mapreduce_train[['key', 'duration']]
mapreduce_train['duration'] = mapreduce_train['duration']*60
print mapreduce_train.head()
print mapreduce_train.shape
mapreduce_train.to_csv('/Users/matthewdunn/Dropbox/NYU/Spring2016/BigData/GroupProject/data/mta_train/mapreduce.csv', index=False, header=False)

In [ ]:
bus_train_map_reduce = pd.concat([mapreduce_train, mapreduce], ignore_index=True)

In [ ]:
print bus_train_map_reduce.head()
print bus_train_map_reduce.shape
bus_train_map_reduce.to_csv('/Users/matthewdunn/Dropbox/NYU/Spring2016/BigData/GroupProject/data/bus_train_mapreduce.csv', index=False, header=False)